Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

In [1]:
# Import Statements
import os
import re

import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import spacy

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

In [2]:
# You may need to change the path
train = pd.read_csv('whiskey-reviews-ds24/train.csv')
test = pd.read_csv('whiskey-reviews-ds24/test.csv')

### Load Competition Data

In [3]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


### Define Pipeline Components

In [4]:
train.description[10]

'\nThis has the classic sweetish, oily, floral, leathery nose of 100% rye grain whisky. A luxurious, creamy, medium-weight dram with caraway rye bread, restrained but hot peppery spices, vague citrus notes, and hints of cedar setting goal posts for punts of sweet and vaguely floral barley sugar, nail polish (in a good way), caramel, barrel wood, well-aged lumber, dark rye bread, ripe apples, milk chocolate, and baking spices. $90 CAD'

In [5]:
# clean up function
def clean_descriptions(txt):
    """
    Takes in text, removes end of line character from text.
    Returns text with end of line characters removed.
    """
    
    clean_txt = re.sub('\\n', '', txt)
    clean_txt = re.sub('[^a-zA-Z 0-9]', '', clean_txt)

    return clean_txt

In [6]:
train['description'] = train['description'].apply(clean_descriptions)
train.head()

,id,description,ratingCategory
0,1321,Sometimes when whisky is batched a few leftove...,1
1,3861,An uncommon exclusive bottling of a 6 year old...,0
2,655,This release is a port version of Amruts Inter...,1
3,555,This 41 year old single cask was aged in a she...,1
4,1965,Quite herbal on the nose with aromas of dried ...,1


In [7]:
X = train['description']
y = train['ratingCategory']

In [8]:
vect = TfidfVectorizer(stop_words='english')
clf = RandomForestClassifier(random_state=42)

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [9]:
params = {
    'vect__max_df': (0.80, 0.85, .90),
    'vect__min_df': (.05, .10), 
    'vect__max_features': (3000, 3500, 3750, 4000), 
    'clf__n_estimators': (10, 100, 150), 
    'clf__max_depth': (None, 20, 40)
}

# create gridsearch object
gs = GridSearchCV(pipe, params, cv=4, n_jobs=-1, verbose=10)

gs.fit(X, y)

Fitting 4 folds for each of 216 candidates, totalling 864 fits


GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(stop_words='english')),
                                       ('clf',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'clf__max_depth': (None, 20, 40),
                         'clf__n_estimators': (10, 100, 150),
                         'vect__max_df': (0.8, 0.85, 0.9),
                         'vect__max_features': (3000, 3500, 3750, 4000),
                         'vect__min_df': (0.05, 0.1)},
             verbose=10)

In [10]:
# get baseline
baseline_acc = y.sum()/len(y)
baseline_acc

0.7367262050403719

In [11]:
gs.best_score_

0.7183754655176711

In [12]:
gs.best_params_

{'clf__max_depth': 40,
 'clf__n_estimators': 100,
 'vect__max_df': 0.8,
 'vect__max_features': 3000,
 'vect__min_df': 0.05}

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [13]:
# Predictions on test sample
pred = gs.predict(test['description'])

In [14]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [15]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [16]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022 entries, 0 to 1021
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   id              1022 non-null   int64
 1   ratingCategory  1022 non-null   int64
dtypes: int64(2)
memory usage: 16.1 KB


In [17]:
subNumber = 0

In [18]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'whiskey-reviews-ds24/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 80% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [19]:
# create data transformers
svd = TruncatedSVD()

tfidf = TfidfVectorizer(stop_words='english')

# create classifier
rfc = RandomForestClassifier(random_state=42)

# instantiate pipeline object
lsa_pipe = Pipeline([('vect', tfidf), # data transformer
                ('svd', svd)])   # data transformer 2

pipe = Pipeline([('lsa', lsa_pipe),   # data transformer
                 ('clf', rfc)])  # classifier

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [20]:
params = {
    "lsa__svd__n_components": (10, 100), 
    "lsa__vect__max_df": (0.80, 0.85, 0.90),
    "clf__n_estimators": (10, 100), 
    "clf__max_depth": (None, 10)  
                               }

gs_lsa = GridSearchCV(pipe, params, cv=5, n_jobs=4, verbose=1)
gs_lsa.fit(X, y)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('lsa',
                                        Pipeline(steps=[('vect',
                                                         TfidfVectorizer(stop_words='english')),
                                                        ('svd',
                                                         TruncatedSVD())])),
                                       ('clf',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=4,
             param_grid={'clf__max_depth': (None, 10),
                         'clf__n_estimators': (10, 100),
                         'lsa__svd__n_components': (10, 100),
                         'lsa__vect__max_df': (0.8, 0.85, 0.9)},
             verbose=1)

In [21]:
gs_lsa.best_score_

0.7247392661445506

In [22]:
gs_lsa.best_params_

{'clf__max_depth': 10,
 'clf__n_estimators': 10,
 'lsa__svd__n_components': 100,
 'lsa__vect__max_df': 0.8}

### Make a Submission File

In [23]:
# Predictions on test sample
pred = gs_lsa.predict(test['description'])

In [24]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [25]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [26]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'whiskey-reviews-ds24/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [27]:
# Apply to your Dataset

# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.ensemble import GradientBoostingClassifier

# from scipy.stats import randint

In [35]:
nlp = spacy.load('en_core_web_lg')

def get_word_vectors(docs):
    """
    This serves as both our tokenizer and vectorizer.
    Returns a nested list of word vectors (i.e. our doc-term matrix).
    """
    return [nlp(doc).vector for doc in docs]

In [36]:
# use our function to create our doc vectors 
X_train_vect = get_word_vectors(X)

In [37]:
rfc = RandomForestClassifier(random_state=42)

embedding_pipe = Pipeline([('clf', rfc)])

In [46]:
param_dist = {
    
    'clf__max_depth' : (None, 10, 40),
    'clf__n_estimators': (100, 150, 200),
    'clf__max_features': ('auto', 0.5, 0.8)
}

gs_embedding = GridSearchCV(embedding_pipe, param_dist, cv=3, n_jobs=-1, verbose=15)

gs_embedding.fit(X_train_vect, y)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('clf',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'clf__max_depth': (None, 10, 40),
                         'clf__max_features': ('auto', 0.5, 0.8),
                         'clf__n_estimators': (100, 150, 200)},
             verbose=15)

In [47]:
gs_embedding.best_score_

0.7403965870971473

In [48]:
# THIS ONE IS MY BEST SUBMISSION --- SAVED AS submission3.csv
gs_embedding.best_params_

{'clf__max_depth': None, 'clf__max_features': 0.5, 'clf__n_estimators': 200}

### Make a Submission File

In [49]:
test['description'] = test['description'].apply(clean_descriptions)
X_test_clean = test.description
X_test_vect = get_word_vectors(X_test_clean)

In [50]:
# Predictions on test sample
pred = gs_embedding.predict(X_test_vect)

In [51]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [52]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [55]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'whiskey-reviews-ds24/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?

1. Sentiment Analysis
- Sentiment Analysis is using NLP, text analysis, and other technologies to identify, extract, quantify, and study people's moods and subjective information.
- Document Classification is not the same thing as Sentiment Analysis. Document Classification  "labels" documents by topic, whereas Sentiment Classification does more than just labelling.
- We classify words as positive, negative, or neutral. We could also use Lexicon, which already has words labelled.
- Common applications of Sentiment Analysis are monitoring social media and to analyze customer feedback.